# Load related library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)


In [2]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

# Load dataset

In [3]:
trainSet = pd.read_csv("./Data/train.csv", header=0)

# Data cleaning

## Drop useless columns

In [4]:
train = trainSet.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])

## Drop rows with missing values

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  int64  
 1   Pclass    100000 non-null  int64  
 2   Sex       100000 non-null  object 
 3   Age       96708 non-null   float64
 4   SibSp     100000 non-null  int64  
 5   Parch     100000 non-null  int64  
 6   Fare      99866 non-null   float64
 7   Embarked  99750 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 6.1+ MB


In [6]:
train = train.dropna(axis=0)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96332 entries, 2 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  96332 non-null  int64  
 1   Pclass    96332 non-null  int64  
 2   Sex       96332 non-null  object 
 3   Age       96332 non-null  float64
 4   SibSp     96332 non-null  int64  
 5   Parch     96332 non-null  int64  
 6   Fare      96332 non-null  float64
 7   Embarked  96332 non-null  object 
dtypes: float64(2), int64(4), object(2)
memory usage: 6.6+ MB


## Dummy coding of the categorical variables

In [10]:
train = pd.get_dummies(train)

# Train, validation data set split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns="Survived", axis=0)
                                                , train['Survived']
                                                , test_size=0.2
                                                , random_state=2023
                                                , stratify=train['Survived'])

# Create function

In [12]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

# Set parameters

In [13]:
params_nn = {
    'neurons': (10, 100)
  , 'activation': (0, 9) # `activationL` has 10 elements
  , 'optimizer': (0, 7) # `optimizerL` has 8 elements
  , 'learning_rate': (0.01, 1)
  , 'batch_size': (200, 1000)
  , "epochs": (20, 100)
}

# Run Bayesian Optimization

In [14]:
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=2023)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------


2023-02-10 23:19:24.556171: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


| 1         | 0.7602    | 2.898     | 912.3     | 67.04     | 0.1353    | 22.72     | 3.275     |
| 2         | 0.5719    | 0.1988    | 781.8     | 61.95     | 0.5495    | 51.07     | 3.51      |
| 3         | 0.5431    | 3.55      | 320.9     | 48.87     | 0.1705    | 40.42     | 1.262     |
| 4         | 0.4856    | 3.519     | 228.5     | 65.19     | 0.2114    | 38.85     | 2.636     |
| 5         | 0.5719    | 1.656     | 283.2     | 56.39     | 0.2039    | 44.07     | 6.514     |
| 6         | 0.5719    | 6.841     | 816.6     | 67.74     | 0.7937    | 82.93     | 6.864     |
| 7         | 0.6995    | 7.963     | 287.8     | 85.58     | 0.3145    | 33.53     | 2.84      |
| 8         | 0.5144    | 4.981     | 700.4     | 26.3      | 0.9726    | 47.02     | 5.052     |
| 9         | 0.4856    | 5.97      | 374.6     | 34.97     | 0.7325    | 87.7      | 2.742     |
| 10        | 0.5719    | 0.9904    | 930.2     | 48.56     | 0.4188    | 26.52     | 4.102     |
| 11        | 0.6551

## Best hyperparameters

In [15]:
params_nn_ = nn_bo.max['params']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]

optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
params_nn_['optimizer'] = optimizerL[round(params_nn_['optimizer'])]
params_nn_

{'activation': 'elu',
 'batch_size': 901.2735443367724,
 'epochs': 39.38147955457231,
 'learning_rate': 0.1512058724048746,
 'neurons': 84.28502005730954,
 'optimizer': 'Adadelta'}